# Usage

Following: https://omegaconf.readthedocs.io/en/latest/usage.html

## Installation

```sh
pip install omegaconf
```

(!) Python 3.6+

## Creating

You can create OmegaConf objects from multiple sources.

### Empty

In [1]:
from omegaconf import OmegaConf

conf = OmegaConf.create()

print(OmegaConf.to_yaml(conf))

{}



### From a dictionary

In [3]:
conf = OmegaConf.create({"k": "v", "list": [1, {"a": "1", "b": "2", 3: "c"}]})

print(OmegaConf.to_yaml(conf))

# NOTE how yaml handles the list below.

k: v
list:
- 1
- a: '1'
  b: '2'
  3: c



In [4]:
# Here is an example of various supported key types:

from enum import Enum


class Color(Enum):
    RED = 1
    BLUE = 2


conf = OmegaConf.create(
    {
        "key": "str",
        123: "int",
        True: "bool",
        3.14: "float",
        Color.RED: "Color",
        b"123": "bytes",
    }
)

print(conf)

{'key': 'str', 123: 'int', True: 'bool', 3.14: 'float', <Color.RED: 1>: 'Color', b'123': 'bytes'}


> OmegaConf supports `str`, `int`, `bool`, `float`, `bytes`, and `Enum` as **dictionary key types**.

### From a list

In [5]:
conf = OmegaConf.create([1, {"a": 10, "b": {"a": 10, 123: "int_key"}}])

print(OmegaConf.to_yaml(conf))

- 1
- a: 10
  b:
    a: 10
    123: int_key



### From a YAML file

In [6]:
conf = OmegaConf.load("source/example.yaml")

# Output is identical to the YAML file
print(OmegaConf.to_yaml(conf))

server:
  port: 80
log:
  file: ???
  rotation: 3600
users:
- user1
- user2



### From a YAML string

In [7]:
s = """
a: b
b: c
list:
- item1
- item2
123: 456
"""

conf = OmegaConf.create(s)

print(OmegaConf.to_yaml(conf))

a: b
b: c
list:
- item1
- item2
123: 456



### From a dot-list

In [8]:
dot_list = ["a.aa.aaa=1", "a.aa.bbb=2", "a.bb.aaa=3", "a.bb.bbb=4"]

conf = OmegaConf.from_dotlist(dot_list)

print(OmegaConf.to_yaml(conf))

a:
  aa:
    aaa: 1
    bbb: 2
  bb:
    aaa: 3
    bbb: 4



### From command line arguments

In [9]:
# Simulating command line arguments
import sys

sys.argv = ["your-program.py", "server.port=82", "log.file=log2.txt"]

conf = OmegaConf.from_cli()

print(OmegaConf.to_yaml(conf))

server:
  port: 82
log:
  file: log2.txt



### From structured config

ℹ️ You can create OmegaConf objects from *structured config classes or objects*.

This provides static and runtime type safety.

See [Structured Configs](https://omegaconf.readthedocs.io/en/latest/structured_config.html) for more details, or keep reading for a minimal example.

In [10]:
from dataclasses import dataclass


@dataclass
class MyConfig:  # NOTE.
    port: int = 80
    host: str = "localhost"


# ⚠️ For strict typing purposes, prefer OmegaConf.structured() when creating structured configs
conf = OmegaConf.structured(MyConfig)  # NOTE: From class.

print(OmegaConf.to_yaml(conf))

port: 80
host: localhost



In [11]:
# You can use an object to initialize the config as well.

conf = OmegaConf.structured(MyConfig(port=443))  # NOTE: From object.

print(OmegaConf.to_yaml(conf))

port: 443
host: localhost



In [12]:
# OmegaConf objects constructed from Structured classes provide runtime type safety.

conf.port = 42  # Ok, type matches
conf.port = "1080"  # Ok! "1080" can be converted to an int

try:
    conf.port = "oops"  # "oops" cannot be converted to an int
except Exception as e:
    print(e)

Value 'oops' of type 'str' could not be converted to Integer
    full_key: port
    object_type=MyConfig


In addition, the config class can be *used as type annotation* for **static type checkers or IDEs**:

```python
def foo(conf: MyConfig):
    print(conf.port) # passes static type checker
    print(conf.pork) # fails static type checker
```

## Access and manipulation

Input YAML file for this section:
```yaml
server:
  port: 80
log:
  file: ???
  rotation: 3600
users:
  - user1
  - user2
```

Or, `source/example.yaml`.

In [14]:
conf = OmegaConf.load("./source/example.yaml")

### Access

In [15]:
# object style access of dictionary elements
conf.server.port

80

In [16]:
# dictionary style access
conf["log"]["rotation"]  # pyright: ignore

3600

In [17]:
# items in list
conf.users[0]

'user1'

### Default values

You can provide default values directly in the accessing code:

In [18]:
conf.get("missing_key", "a default value")  # pyright: ignore

'a default value'

### Mandatory values

Use the value `???` to indicate parameters that need to be set prior to access:

In [19]:
try:
    print(conf.log.file)
except Exception as e:
    print(e)

Missing mandatory value: log.file
    full_key: log.file
    object_type=dict


### Manipulation

In [20]:
# Changing existing keys
conf.server.port = 81

conf.server.port

81

In [21]:
# Adding new keys
conf.server.hostname = "localhost"

conf.server.hostname

'localhost'

In [22]:
# Adding a new dictionary
conf.database = {"hostname": "database01", "port": 3306}

conf.database

{'hostname': 'database01', 'port': 3306}

## Serialization

OmegaConf objects can be saved and loaded with `OmegaConf.save()` and `OmegaConf.load()`.

The created file is in `YAML` format.

Save and load can operate on *file-names*, `Path`s and *file objects*.

### Save/Load YAML file

In [23]:
import tempfile

conf = OmegaConf.create({"foo": 10, "bar": 20, 123: 456})

with tempfile.NamedTemporaryFile() as fp:
    OmegaConf.save(config=conf, f=fp.name)
    loaded = OmegaConf.load(fp.name)
    assert conf == loaded

> ⚠️ Note that this **does not retain type information**.

### Save/Load pickle file

Use `pickle` to save and load while retaining the type information.

> ⚠️ Note that the saved file may be incompatible across different versions of OmegaConf.

In [24]:
import pickle

conf = OmegaConf.create({"foo": 10, "bar": 20, 123: 456})

with tempfile.TemporaryFile() as fp:
    pickle.dump(conf, fp)

    fp.flush()
    assert fp.seek(0) == 0

    loaded = pickle.load(fp)
    assert conf == loaded

> Note for Python3.6 users: due to limitations in pickling support, structured configs with complex type hints
> (such as nested container types or containers with optional element types) cannot be pickled using Python3.6.

## Variable interpolation

OmegaConf supports variable interpolation.

Interpolations are *evaluated lazily* on access.

### Config node interpolation

The interpolated variable can be the path to another node in the configuration, and in that case the value will be the value of that node.

This path may use either dot-notation (`foo.1`), brackets (`[foo][1]`) or a mix of both (`foo[1]`, `[foo].1`).

Interpolations are *absolute* by default.

Relative interpolation are prefixed by one or more **dots**: The first dot denotes the level of the node itself and
additional dots are going up the parent hierarchy. e.g. `${..foo}` points to the foo sibling of the parent of the current node.

**⚠️ NOTE: Interpolations may cause config cycles. Such cycles are forbidden and may cause undefined behavior.**

Input YAML file:
```yaml
server:
  host: localhost
  port: 80

client:
  url: http://${server.host}:${server.port}/
  server_port: ${server.port}
  # relative interpolation
  description: Client of ${.url}
```

In [25]:
conf = OmegaConf.load("./source/config_interpolation.yaml")


def show(x):
    print(f"type: {type(x).__name__}, value: {repr(x)}")

In [26]:
# Primitive interpolation types are inherited from the reference

show(conf.client.server_port)

type: int, value: 80


In [27]:
# String interpolations concatenate fragments into a string

show(conf.client.url)

type: str, value: 'http://localhost:80/'


In [28]:
# Relative interpolation example

show(conf.client.description)

type: str, value: 'Client of http://localhost:80/'


### Nested interpolation

Interpolations may be nested, enabling more advanced behavior like dynamically selecting a sub-config:

In [29]:
cfg = OmegaConf.create(
    {
        "plans": {
            "A": "plan A",
            "B": "plan B",
        },
        "selected_plan": "A",
        "plan": "${plans[${selected_plan}]}",  # NOTE.
    }
)

In [30]:
cfg.plan  # default plan

'plan A'

In [31]:
cfg.selected_plan = "B"

cfg.plan  # new plan

'plan B'

In [32]:
# Interpolated nodes can be any node in the config, not just leaf nodes:

cfg = OmegaConf.create(
    {
        "john": {"height": 180, "weight": 75},
        "player": "${john}",
    }
)

(cfg.player.height, cfg.player.weight)

(180, 75)

### Resolvers

Add **new interpolation types** by registering resolvers using `OmegaConf.register_new_resolver()`.

Such resolvers are called when the config node is accessed.

The minimal example below shows its most basic usage, see [Resolvers](https://omegaconf.readthedocs.io/en/latest/custom_resolvers.html) for more details.

In [33]:
OmegaConf.register_new_resolver("add", lambda *numbers: sum(numbers))

c = OmegaConf.create({"total": "${add:1,2,3}"})

c.total

6

### Built-in resolvers

OmegaConf comes with a set of built-in custom resolvers:
* `oc.create`: Dynamically generating config nodes
* `oc.decode`: Parsing an input string using interpolation grammar
* `oc.deprecated`: Deprecate a key in your config
* `oc.env`: Accessing environment variables
* `oc.select`: Selecting an interpolation key, similar to interpolation but more flexible
* `oc.dict`.{keys,value}: Viewing the keys or the values of a dictionary as a list

Links to each: https://omegaconf.readthedocs.io/en/latest/usage.html#built-in-resolvers

## Merging configurations

Merging configurations enables the creation of reusable configuration files **for each logical component**
instead of a single config file for each variation of your task.


### `OmegaConf.merge()`

Machine learning experiment example:
```python
conf = OmegaConf.merge(base_cfg, model_cfg, optimizer_cfg, dataset_cfg)
```

Web server configuration example:
```python
conf = OmegaConf.merge(server_cfg, plugin1_cfg, site1_cfg, site2_cfg)
```

The following example creates *two* configs from *files*, and *one* from the *cli*.

It then *combines* them into a single object.

Note how the port changes to 82, and how the users lists are combined.

In [34]:
from omegaconf import OmegaConf
import sys

In [35]:
# Simulate command line arguments

sys.argv = ["program.py", "server.port=82"]

In [36]:
base_conf = OmegaConf.load("source/example2.yaml")
second_conf = OmegaConf.load("source/example3.yaml")

cli_conf = OmegaConf.from_cli()
cli_conf

{'server': {'port': 82}}

In [37]:
# merge them all

conf = OmegaConf.merge(base_conf, second_conf, cli_conf)

print(OmegaConf.to_yaml(conf))

server:
  port: 82
users:
- user1
- user2
log:
  file: log.txt



### `OmegaConf.unsafe_merge()`

OmegaConf offers a second *faster* function to merge config objects:
```python
conf = OmegaConf.unsafe_merge(base_cfg, model_cfg, optimizer_cfg, dataset_cfg)
```

> Unlike `OmegaConf.merge()`, `unsafe_merge()` is **destroying the input configs** and *they should no longer be used* after this call.
>
> The upside is that it’s substantially faster.

## Configuration flags

OmegaConf support several **configuration flags**.

Configuration flags can be set on any configuration node (`Sequence` or `Mapping`).

If a configuration flag is not set it *inherits the value from the parent* of the node.

The default value inherited from the root node is always false.

### Read-only flag

A read-only configuration cannot be modified.

An attempt to modify it will result in `omegaconf.ReadonlyConfigError` exception.

In [38]:
conf = OmegaConf.create({"a": {"b": 10}})

OmegaConf.set_readonly(conf, True)  # NOTE.

try:
    conf.a.b = 20
except Exception as e:
    print(e)

Cannot change read-only config container
    full_key: a.b
    object_type=dict


ℹ️ Useful!

You can temporarily remove the read only flag from a config object:

In [40]:
from omegaconf import read_write  # NOTE.

conf = OmegaConf.create({"a": {"b": 10}})
OmegaConf.set_readonly(conf, True)

with read_write(conf):  # NOTE.
    conf.a.b = 20

conf

{'a': {'b': 20}}

### Struct flag

⚠️ By default, OmegaConf dictionaries allow write access to unknown fields.

If a field does not exist:
* writing it will *create the field*,
* and attempting to *access the field* before creation will raise an *exception* (either `ConfigKeyError` or `ConfigAttributeError`, depending on the mode of access).

It’s sometime useful to change this behavior.

Using `OmegaConf.set_struct`, it is possible to **prevent the creation of fields that do not exist**:

In [42]:
conf = OmegaConf.create({"a": {"aa": 10, "bb": 20}})
OmegaConf.set_struct(conf, True)  # NOTE.

try:
    conf.a.cc = 30  # Trying to set a field that doesn't exist.
except Exception as e:
    print(e)

Key 'cc' is not in struct
    full_key: a.cc
    object_type=dict


ℹ️ You can temporarily remove the struct flag from a config object:

In [43]:
from omegaconf import open_dict  # NOTE.

conf = OmegaConf.create({"a": {"aa": 10, "bb": 20}})
OmegaConf.set_struct(conf, True)

with open_dict(conf):
    conf.a.cc = 30

conf.a.cc

30

## Utility functions

### `OmegaConf.to_container`

OmegaConf config objects *looks very similar to python `dict` and `list`*, **but in fact are not**.

Use `OmegaConf.to_container(cfg: Container, resolve: bool)` to convert to a primitive container.

If `resolve` is set to `True`, *interpolations will be resolved during conversion*.

In [44]:
from omegaconf import DictConfig

conf = OmegaConf.create({"foo": "bar", "foo2": "${foo}"})
assert type(conf) == DictConfig

primitive = OmegaConf.to_container(conf)
show(primitive)

type: dict, value: {'foo': 'bar', 'foo2': '${foo}'}


In [46]:
resolved = OmegaConf.to_container(conf, resolve=True)
show(resolved)

type: dict, value: {'foo': 'bar', 'foo2': 'bar'}


### Using `throw_on_missing`

You can control how **missing values** are handled by `OmegaConf.to_container()` using the `throw_on_missing` keyword argument.

In [48]:
conf = OmegaConf.create({"foo": "bar", "missing": "???"})

has_missing = OmegaConf.to_container(conf, throw_on_missing=False)

show(has_missing)  # NOTE: Missing item(s) will just take the value "???"

type: dict, value: {'foo': 'bar', 'missing': '???'}


In [49]:
# But this will throw an exception.

try:
    OmegaConf.to_container(conf, throw_on_missing=True)  # NOTE.
except Exception as e:
    print(e)

Missing mandatory value: missing
    full_key: missing
    object_type=dict


By default, `throw_on_missing=False`.

Setting `throw_on_missing=True` can be useful if you want your program to fail fast when there are missing values in the config.

### Using `structured_config_mode`

You can customize the treatment of `OmegaConf.to_container()` for *Structured Config nodes* using the `structured_config_mode` option.

1.
The default, `structured_config_mode=SCMode.DICT`, converts Structured Config nodes **to plain dict**.

2.
Using `structured_config_mode=SCMode.DICT_CONFIG` causes such nodes to **remain as `DictConfig`**,
allowing attribute style access on the resulting node.

3.
Using `structured_config_mode=SCMode.INSTANTIATE`, Structured Config nodes are *converted to instances of the backing dataclass or attrs class*.

(!) Note that when `structured_config_mode=SCMode.INSTANTIATE`, interpolations nested within a structured config node will be resolved, even if `OmegaConf.to_container` is called with the the keyword argument `resolve=False`, so that interpolations are resolved before being used to instantiate dataclass/attr class instances. Interpolations within non-structured parent nodes will be resolved (or not) as usual, according to the resolve keyword arg.

In [50]:
from omegaconf import SCMode

conf = OmegaConf.create(
    {"structured_config": MyConfig}
)  # MyConfig from earlier in tutorial.

container = OmegaConf.to_container(
    conf, structured_config_mode=SCMode.DICT_CONFIG  # NOTE.
)

show(container)

type: dict, value: {'structured_config': {'port': 80, 'host': 'localhost'}}


In [51]:
show(container["structured_config"])  # pyright: ignore

type: DictConfig, value: {'port': 80, 'host': 'localhost'}


### `OmegaConf.to_object`

The `OmegaConf.to_object` method recursively converts `DictConfig` and `ListConfig` objects into plain Python `dict`s and `list`s,
with the *exception* that Structured Config objects are converted into *instances* of the *backing `dataclass` or `attr` class*.

Interpolations in the config are *always resolved* by `OmegaConf.to_object`.

In [52]:
container = OmegaConf.to_object(conf)

show(container)

# NOTE it is a `dict`

type: dict, value: {'structured_config': MyConfig(port=80, host='localhost')}


In [53]:
show(container["structured_config"])  # pyright: ignore

# NOTE it is a `MyConfig`!

type: MyConfig, value: MyConfig(port=80, host='localhost')


Note that here, `container["structured_config"]` is actually an instance of `MyConfig`,
whereas in the previous examples we had a `dict` or a `DictConfig` object that was **duck-typed to look like an instance of `MyConfig`**.

> The call `OmegaConf.to_object(conf)` is **equivalent** to:
> 
> `OmegaConf.to_container(conf, resolve=True, throw_on_missing=True, structured_config_mode=SCMode.INSTANTIATE)`.

### `OmegaConf.resolve`

```python
def resolve(cfg: Container) -> None:
    """
    Resolves all interpolations in the given config object in-place.
    :param cfg: An OmegaConf container (DictConfig, ListConfig)
                Raises a ValueError if the input object is not an OmegaConf container.
    """
```

Normally interpolations are resolved *lazily*, at access time.

This function *eagerly* resolves all interpolations in the given config object in-place. Example:

In [54]:
cfg = OmegaConf.create({"a": 10, "b": "${a}"})

show(cfg)

type: DictConfig, value: {'a': 10, 'b': '${a}'}


In [55]:
assert cfg.a == cfg.b == 10  # lazily resolving interpolation

In [56]:
OmegaConf.resolve(cfg)

show(cfg)

type: DictConfig, value: {'a': 10, 'b': 10}


### `OmegaConf.select`

`OmegaConf.select()` allows you to select a config node or value, using either a dot-notation or brackets to denote sub-keys.

In [62]:
cfg = OmegaConf.create(
    {
        "foo": {
            "missing": "???",
            "bar": {
                "zonk": 10,
            },
        }
    }
)

assert OmegaConf.select(cfg, "foo") == {
    "missing": "???",
    "bar": {
        "zonk": 10,
    },
}

assert OmegaConf.select(cfg, "foo.bar") == {
    "zonk": 10,
}

assert OmegaConf.select(cfg, "foo.bar.zonk") == 10  # dots

assert OmegaConf.select(cfg, "foo[bar][zonk]") == 10  # brackets

assert OmegaConf.select(cfg, "no_such_key", default=99) == 99

assert OmegaConf.select(cfg, "foo.missing") is None

assert OmegaConf.select(cfg, "foo.missing", default=99) == 99

In [63]:
try:
    OmegaConf.select(cfg, "foo.missing", throw_on_missing=True)
except Exception as e:
    print(e)

Missing mandatory value: foo.missing
    full_key: foo.missing
    object_type=dict


### `OmegaConf.update`

`OmegaConf.update()` allows you to **update values** in your config using either:
* a dot-notation or 
* brackets to denote sub-keys.

The `merge` flag controls the behavior if the input is a `dict` or a `list`.

If `merge=True` true (the default), `dict`s and `list`s are *merged* instead of being *assigned*.

The `force_add` flag ensures that the *path is created* even if it will result in insertion of new values into struct nodes.

In [64]:
cfg = OmegaConf.create({"foo": {"bar": 10}})

OmegaConf.update(cfg, "foo.bar", 20)
assert cfg.foo.bar == 20

# Set dictionary value (using dot notation)
OmegaConf.update(cfg, "foo.bar", {"zonk": 30}, merge=False)
assert cfg.foo.bar == {"zonk": 30}

# Merge dictionary value (using bracket notation)
# note that merge is True by default, so you don't really need it here.
OmegaConf.update(cfg, "foo[bar]", {"oompa": 40}, merge=True)
assert cfg.foo.bar == {"zonk": 30, "oompa": 40}

# force_add ignores nodes in struct mode or Structured Configs nodes
# and updates anyway, inserting keys as needed.
OmegaConf.set_struct(cfg, True)
OmegaConf.update(cfg, "a.b.c.d", 10, force_add=True)

assert cfg.a.b.c.d == 10

### `OmegaConf.masked_copy`

Creates a copy of a DictConfig that *contains only specific keys*.

In [65]:
conf = OmegaConf.create({"a": {"b": 10}, "c": 20})

print(OmegaConf.to_yaml(conf))

a:
  b: 10
c: 20



In [66]:
c = OmegaConf.masked_copy(conf, ["a"])

print(OmegaConf.to_yaml(c))

a:
  b: 10



### `OmegaConf.is_missing`

Tests if a value is missing (`"???"`).

In [67]:
cfg = OmegaConf.create({"foo": 10, "bar": "???"})

assert not OmegaConf.is_missing(cfg, "foo")
assert OmegaConf.is_missing(cfg, "bar")

### `OmegaConf.is_interpolation`

Tests if a value is an interpolation.

In [69]:
cfg = OmegaConf.create({"foo": 10, "bar": "${foo}"})

assert not OmegaConf.is_interpolation(cfg, "foo")
assert OmegaConf.is_interpolation(cfg, "bar")

### `OmegaConf.{is_config, is_dict, is_list}`

* `OmegaConf.is_config` tests whether an object is an OmegaConf object (e.g. `DictConfig` or `ListConfig`).
* `OmegaConf.is_dict(cfg)` is equivalent to `isinstance(cfg, DictConfig)`, and
* `OmegaConf.is_list(cfg)` is equivalent to `isinstance(cfg, ListConfig)`.

In [70]:
# dict:

d = OmegaConf.create({"foo": "bar"})

assert OmegaConf.is_config(d)
assert OmegaConf.is_dict(d)
assert not OmegaConf.is_list(d)

# list:

l = OmegaConf.create([1, 2, 3])

assert OmegaConf.is_config(l)
assert OmegaConf.is_list(l)
assert not OmegaConf.is_dict(l)

### `OmegaConf.missing_keys`

`OmegaConf.missing_keys(cfg)` returns a set of missing keys present in the input `cfg`.

Each missing key is *represented as a str, using a dotlist style*.

This utility function can be used after creating a config object, after merging sources and so on, to check for missing mandatory fields and aid in creating a proper error message.

In [71]:
missings = OmegaConf.missing_keys(
    {"foo": {"bar": "???"}, "missing": "???", "list": ["a", None, "???"]}
)

assert missings == {"list[2]", "foo.bar", "missing"}